<div class="alert alert-block alert-info" style="margin-top: 10px">
<a id="ref1"></a>
<center>1 学习笔记</center>
</div>

## 6.1	贝叶斯分类器与多分类任务

## 6.2 贝叶斯分类器的基本原理及极大似然估计的作用
### 6.2.1 贝叶斯分类器的基本原理
贝叶斯分类器的分类原理是通过某对象的先验概率P(c)，利用贝叶斯公式计算出其后验概率，即该对象属于某一类的概率，选择具有最大后验概率的类作为对象所属的类。
### 6.2.2 极大似然估计的作用
极大似然估计的作用是估计类条件概率P(x|c)，根据数据采样估计概率分布参数。

## 6.3	朴素贝叶斯分类器与半朴素贝叶斯分类器的原理与应用

### 6.3.1 朴素贝叶斯算法
朴素贝叶斯法是基于贝叶斯定理与特征条件独立假设的分类方法。对于给定的训练数据集，首先基于特征条件独立假设学习输入/输出的联合概率分布，然后基于此模型，对给定的输入 x ，利用贝叶斯定理，利用贝叶斯定理求出后验概率最大的输出 y

<p>输入：训练数据$D=\left\{(x_1,y_1),(x_2,y_2),\cdots,(x_N,y_N)\right\}$，其中$x_i=(x_i^{(1)},x_i^{(2)},\cdots,x_i^{(n)})$，$x_i^{(j)}$是第$i$个样本的第$j$个特征，$x_i^{(j)}\in\left\{a_{j1},a_{j2},\cdots,a_{jS_j}\right\}^T$，$a_{jl}$是第$j$个特征可能取的第$l$个值，$j=1,2,\cdots,n$，$l=1,2,\cdots,S_j$，$y_i\in\left\{c_1,c_2,\cdots,c_K\right\}$；实例$x=(x^{(1)},x^{(2)},\cdots,x^{(n)})^T$</p>
<p>输出：实例$x$的分类</p>
<p>算法步骤如下：</p>
<ul>
    <li>计算先验概率和条件概率
        $$P(Y=c_k)=\frac{\sum_{i=1}^NI(y_i=c_k)}{N},k=1,2,\cdots,K$$
        $$$$
        $$P(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^NI(x_i^{(j)}=a_{jl},y_i=c_k)}{\sum_{i=1}^NI(y_i=c_k)}$$
        $$$$
        $$j=1,2,\cdots,n;l=1,2,\cdots,S_j;k=1,2,\cdots,K$$</li><br>
    <li>对于给定的实例$x=(x^{(1)},x^{(2)},\cdots,x^{(n)})^T$，计算：$$P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k),k=1,2,\cdots,K$$</li><br>
    <li>确定实例$x$的类$$y=\arg\max_{c_k}P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k),k=1,2,\cdots,K$$</li>

### 6.3.2 半朴素贝叶斯算法
半朴素贝叶斯法不同于朴素贝叶斯法，它对特征条件独立性假设进行了一定程度的放松。基本想法是适当考虑一部分特征间的相互信息，从而既不需进行完全联合概率计算，又不至于彻底忽略了比较强的依赖关系。
<p>半朴素贝叶斯分类起常用的策略为One-Dependent Estimator，即假设每个属性在类别之外最多仅依赖于一个其他属性，即：</p>

$$P(c|x)\alpha P(c) \prod_{i=1}^{d}P( x_i  |c, pa_i )$$

<p>其中$pa_i$为属性$x_i$所依赖的属性，称为$x_i$的父属性。若父属性$pa_i$已知，则可以采用半朴素贝叶斯算法的方法来估计概率值。至于确定超父属性，最直接的做法是假设所有属性都依赖于同一个属性，然后通过交叉验证等模型选择方法来确定，这种做法我们称之为Super-Parent ODE方法。</p>


<div class="alert alert-block alert-info" style="margin-top: 10px">
<a id="ref2"></a>
<center>2 算法实现</center>
</div>

## 朴素贝叶斯分类器实现

In [27]:
class NaiveBayesClassifier:
    def __init__(self):
        self.PLabels=dict()
        self.NBClassify=dict()        
    
    def train(self,data):
        labels = data[:, -1]
        label_list=list(set(labels))
        PLabels=dict()
        NBClassify=dict()
        for label in label_list:
            PLabels[label]=(sum([1 for l in labels if l==label])+1 )/ len(labels)
            NBClassify[label]=dict()
            
        for label in NBClassify.keys():
            sub_data = data[data[:, -1] == label]
            sub_data = np.array(sub_data)
            for k in range(sub_data.shape[1]):
                NBClassify[label][k] = dict()
                tags = list(set(data[:, k]))
                d = sub_data[:, k]
                for tag in tags:
                    NBClassify[label][k][tag] = (sum([1 for i in d
                    if i == tag])+1)/len(d)
        self.PLabels=PLabels
        self.NBClassify=NBClassify

    def predict(self,data):
        predict_vec = list()
        for sample in data:
            pl=dict()
            for i in self.PLabels.keys():
                pl[i] = np.math.log(self.PLabels[i], 2) 
                #pl[i]=tf.log(self.PLabels[i])
            for label in self.NBClassify.keys():
                for k, tag in enumerate(sample):
                    pl[label] += np.math.log(self.NBClassify[label][k][tag], 2)  
                    #pl[label] += tf.log(self.NBClassify[label][k][tag]) 
            predict_vec.append(max(pl,key=pl.get))
            #predict_vec.append(tf.argmax(pl))
        return np.array(predict_vec)

2）对比分析使用朴素贝叶斯分类器与半朴素贝叶斯分类器编写的天气预报模型的差异。
## 半朴素贝叶斯分类器
<p>这里我们采用了独依赖分类器AODE（Averaged One-Dependent Estimator），即：</p>

$$P(c|x)\alpha \sum_{i=1 }^{d}  P(c) \prod_{i=1}^{d}P( x_i  |c, pa_i ) $$


In [29]:
class SemiNaiveBayesClassifier:
    def __init__(self):
        self.PLabels=dict()
        self.SNBClassify=dict()
    
    def train(self,data):
        labels = data[:, -1]
        label_list=list(set(labels))
        PLabels=dict()
        SNBClassify=dict()
        for label in label_list:
            PLabels[label]=dict()
            sub_data = data[data[:, -1] == label]
            sub_data = np.array(sub_data)
            for k in range(sub_data.shape[1]):
                PLabels[label][k]=dict()
                tags=list(set(data[:,k]))
                d=sub_data[:,k]
                for tag in tags:
                    PLabels[label][k][tag]=(sum([1 for i in d if i==tag])+1 )/ len(data)    
            
        for label in label_list:
            SNBClassify[label]=dict()
            sub_data = data[data[:, -1] == label]
            sub_data = np.array(sub_data)
            for k in range(sub_data.shape[1]):
                SNBClassify[label][k] = dict()
                tags = list(set(data[:, k]))
                for tag in tags:
                    SNBClassify[label][k][tag]=dict()
                    sub_sub_data=sub_data[sub_data[:,k]==tag]
                    if len(sub_sub_data)>=30:
                        for j in range(sub_sub_data.shape[1]):
                            SNBClassify[label][k][tag][j]=dict()
                            stags=list(set(sub_sub_data[:,j]))
                            d=sub_sub_data[:,j]
                            for stag in stags:
                                SNBClassify[label][k][tag][j][stag] = (sum([1 for i in d if i == stag])+1)/(len(sub_sub_data))
        self.PLabels=PLabels
        self.SNBClassify=SNBClassify

    def predict(self,data):
        predict_vec = list()
        for sample in data:
            pl=dict()
            for label in self.PLabels.keys():
                p = list()
                for k, tag in enumerate(sample):
                    pcx=np.math.log(self.PLabels[label][k][tag], 2)
                    for j, jtag in enumerate(sample):
                        if j in self.SNBClassify[label][k][tag].keys():
                            if jtag in self.SNBClassify[label][k][tag][j].keys():
                                pcx+=np.math.log(self.SNBClassify[label][k][tag][j][jtag], 2)
                    p.append(pcx)
                p=np.array(p)
                pl[label]= np.sum(p)        
            predict_vec.append(max(pl,key=pl.get))
        return np.array(predict_vec)